In [1]:
import django
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'FacebookPosts.settings'
django.setup()
from posts.models import User, Post, PostReaction, Comment, CommentReaction

In [118]:
import posts.operations as op
from django.db import connection, reset_queries
from django.db.models import Prefetch
def get_comment_data(comment):
    d = {
        "comment_id":comment['id'],
        "commenter":{
            "user_id":comment['user_id'],
            "name": comment['user__user_name'],
            "profile_pic": comment['user__profile_pic']
        },
        "commented_at": comment['comment_create_date'].strftime("%Y-%m-%d %H:%M:%S"),
        "comment_content": comment['message']
    }
    return d
post = Post.objects.select_related('user').get(id=2)
posted_by = op.get_user_data(post)
posted_at = post.post_created_date.strftime("%Y-%m-%d %H:%M:%S")
post_content = post.post_description
post_reactions = PostReaction.objects.filter(post_id=1).values('reaction')
reactions = []
for reaction in post_reactions:
    reactions.append(reaction['reaction'])
post_reaction_count = len(reactions)
post_reactions = list(set(reactions))

post_comments = Comment.objects.select_related('user').filter(post_id=2, commented_on_id=None).values('id', 'user_id', 'user__user_name', 'user__profile_pic', 'commented_on_id','comment_create_date','message')
comments_ids = [comment['id'] for comment in post_comments]

replies = Comment.objects.select_related('user').filter(commented_on_id__in=comments_ids).values('id', 'user_id', 'user__user_name', 'user__profile_pic', 'commented_on_id','comment_create_date','message')

comments_ids[len(comments_ids)+1:] = [reply['id'] for reply in replies]

comment_reactions = CommentReaction.objects.select_related('user').filter(comment_id__in=comments_ids).values('comment_id','reaction')

comment_reaction = {}
for react in comment_reactions:
    l = []
    r = react['reaction']
    l.append(r)
    try:
        comment_reaction[react['comment_id']].append(r)
        
    except:
        comment_reaction[react['comment_id']] = l

for id in comment_reaction:
    count = len(comment_reaction[id])
    reactions = list(set(comment_reaction[id]))
    data = {"count": count, "type": reactions}
    comment_reaction[id] = data

comment_replies = {}
for reply in replies:
    comment_reply = []
    d = get_comment_data(reply)
    try:
        d["reactions"] = comment_reaction[reply['id']]
    except:
        d["reactions"] = {"count": 0, "type": []}
    comment_reply.append(d)
    try:
        comment_replies[reply['commented_on_id']].append(d)
    except:
        comment_replies[reply['commented_on_id']] = comment_reply

comments = []
for comment in post_comments:
    d = get_comment_data(comment)
    d["reactions"] = comment_reaction[comment['id']]
    d["replies_count"] = len(comment_replies[comment['id']])
    d["replies"]= comment_replies[comment['id']]
    comments.append(d)
comments
    
result = {"post_id": 2, "posted_by": posted_by, "posted_at": posted_at, "post_content": post_content,
              "reactions": {
                  "count": post_reaction_count,
                  "type": post_reactions
              }, "comments": comments, "comments_count": len(comments)}
result

{'post_id': 2,
 'posted_by': {'user_id': 3,
  'name': 'Manoj',
  'profile_pic_url': 'http://manoj.jpg'},
 'posted_at': '2019-06-27 12:28:37',
 'post_content': 'Good Morning Friends',
 'reactions': {'count': 4, 'type': ['LOVE', 'LIKE']},
 'comments': [{'comment_id': 2,
   'commenter': {'user_id': 1,
    'name': 'Karthik',
    'profile_pic': 'https://karthik.jpg'},
   'commented_at': '2019-06-27 13:01:01',
   'comment_content': 'Good Eve',
   'reactions': {'count': 1, 'type': ['LOVE']},
   'replies_count': 2,
   'replies': [{'comment_id': 3,
     'commenter': {'user_id': 3,
      'name': 'Manoj',
      'profile_pic': 'http://manoj.jpg'},
     'commented_at': '2019-06-28 03:33:21',
     'comment_content': 'ABC',
     'reactions': {'count': 1, 'type': ['LIKE']}},
    {'comment_id': 4,
     'commenter': {'user_id': 4,
      'name': 'Sravan',
      'profile_pic': 'http://sravan.png'},
     'commented_at': '2019-06-28 03:33:52',
     'comment_content': 'XYZ',
     'reactions': {'count': 0, 't

In [110]:
qry = connection.queries
qry

[{'sql': 'SELECT "posts_post"."id", "posts_post"."user_id", "posts_post"."post_description", "posts_post"."post_created_date", "posts_user"."id", "posts_user"."user_name", "posts_user"."profile_pic" FROM "posts_post" INNER JOIN "posts_user" ON ("posts_post"."user_id" = "posts_user"."id") WHERE "posts_post"."id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "posts_postreaction"."reaction" FROM "posts_postreaction" WHERE "posts_postreaction"."post_id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "posts_comment"."id", "posts_comment"."user_id", "posts_user"."user_name", "posts_user"."profile_pic", "posts_comment"."commented_on_id_id", "posts_comment"."comment_create_date", "posts_comment"."message" FROM "posts_comment" INNER JOIN "posts_user" ON ("posts_comment"."user_id" = "posts_user"."id") WHERE ("posts_comment"."commented_on_id_id" IS NULL AND "posts_comment"."post_id" = 1)',
  'time': '0.000'},
 {'sql': 'SELECT "posts_comment"."id", "posts_comment"."user_id", "posts_user"."user_name", "po

In [111]:
print(len(qry))
reset_queries()

5


In [5]:
op.get_post(1)

{'post_id': 1,
 'posted_by': {'user_id': 1,
  'name': 'Karthik',
  'profile_pic_url': 'https://karthik.jpg'},
 'posted_at': '2019-06-27 12:28:28',
 'post_content': 'Hi friends Good Evening',
 'reactions': {'count': 4, 'type': ['LOVE', 'LIKE']},
 'comments': [{'comment_id': 1,
   'commenter': {'user_id': 2,
    'name': 'Bharat',
    'profile_pic_url': 'http://bharat.png'},
   'commented_at': '2019-06-27 12:39:38',
   'comment_content': 'Good Evening',
   'reactions': {'count': 1, 'type': ['LOVE']},
   'replies_count': 2,
   'replies': [{'comment_id': 5,
     'commenter': {'user_id': 4,
      'name': 'Sravan',
      'profile_pic_url': 'http://sravan.png'},
     'commented_at': '2019-06-28 03:34:22',
     'comment_content': 'REPLY',
     'reactions': {'count': 1, 'type': ['LIKE']}},
    {'comment_id': 6,
     'commenter': {'user_id': 4,
      'name': 'Sravan',
      'profile_pic_url': 'http://sravan.png'},
     'commented_at': '2019-06-28 03:40:48',
     'comment_content': 'REPLY_reply',


In [6]:
qry = connection.queries
qry

[{'sql': 'SELECT "posts_post"."id", "posts_post"."user_id", "posts_post"."post_description", "posts_post"."post_created_date", "posts_user"."id", "posts_user"."user_name", "posts_user"."profile_pic" FROM "posts_post" INNER JOIN "posts_user" ON ("posts_post"."user_id" = "posts_user"."id") WHERE "posts_post"."id" = 1',
  'time': '0.000'},
 {'sql': 'SELECT "posts_postreaction"."id", "posts_postreaction"."post_id", "posts_postreaction"."user_id", "posts_postreaction"."reaction" FROM "posts_postreaction" WHERE "posts_postreaction"."post_id" IN (1)',
  'time': '0.000'},
 {'sql': 'SELECT "posts_comment"."id", "posts_comment"."post_id", "posts_comment"."user_id", "posts_comment"."commented_on_id_id", "posts_comment"."comment_create_date", "posts_comment"."message" FROM "posts_comment" WHERE ("posts_comment"."commented_on_id_id" IS NULL AND "posts_comment"."post_id" IN (1))',
  'time': '0.000'},
 {'sql': 'SELECT DISTINCT "posts_postreaction"."reaction" FROM "posts_postreaction" WHERE "posts_pos

In [7]:
print(len(qry))
reset_queries()

23


In [8]:
posts = User.objects.prefetch_related('postreaction_set').filter(id=1)
print(posts)
qry = connection.queries
qry

<QuerySet [<User: Karthik>]>


[{'sql': 'SELECT "posts_user"."id", "posts_user"."user_name", "posts_user"."profile_pic" FROM "posts_user" WHERE "posts_user"."id" = 1  LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "posts_postreaction"."id", "posts_postreaction"."post_id", "posts_postreaction"."user_id", "posts_postreaction"."reaction" FROM "posts_postreaction" WHERE "posts_postreaction"."user_id" IN (1)',
  'time': '0.000'}]

In [9]:
print(len(qry))
reset_queries()

2
